In [ ]:
import os
import plaid
import requests
import datetime
import json
import pandas as pd
%matplotlib inline

In [ ]:
def pretty_print_response(response):
  print(json.dumps(response, indent=4, sort_keys=True))

In [ ]:
PLAID_CLIENT_ID = os.getenv('PLAID_CLIENT_ID')
PLAID_SECRET = os.getenv('PLAID_SECRET')
PLAID_PUBLIC_KEY = os.getenv('PLAID_PUBLIC_KEY')
PLAID_ENV = os.getenv('PLAID_ENV','sandbox')
PLAID_PRODUCTS = os.getenv('PLAID_PRODUCTS','transactions')

# Plaid Access Token

In this section, you will use the plaid-python api to generate the correct authentication tokens to access data in the free developer Sandbox. This mimics how you might connect to your own account or a customer account, but due to privacy issues, this homework will only require connecting to and analyzing the fake data from the developer sandbox that Plaid provides. 

Complete the following steps to generate an access token:
1. Create a client to connect to plaid
2. Use the client to generate a public token and request the following items: 
['transactions', 'income', 'assets']
3. Exchange the public token for an access token
4. Test the access token by requesting and printing the available test accounts

### 1. Create a client to connect to plaid

In [ ]:
INSTITUTION_ID = "ins_109508"

In [ ]:

# YOUR CODE HERE
client = plaid.Client(client_id=PLAID_CLIENT_ID, secret=PLAID_SECRET, public_key=PLAID_PUBLIC_KEY, environment='sandbox')

### 2. Generate a public token

In [ ]:
# YOUR CODE HERE
create_tkn_response = client.Sandbox.public_token.create(INSTITUTION_ID, ['transactions','income','assets'])

### 3. Exchange the public token for an access token

In [ ]:
# YOUR CODE HERE
# Exchange public token for access token
exchange_response = client.Item.public_token.exchange(create_tkn_response['public_token'])
# Store access token as variable
access_token = exchange_response['access_token']

### 4. Fetch Accounts

In [ ]:
# YOUR CODE HERE
client.Accounts.get(access_token)

---

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days
2. Print the categories for each transaction type
3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)
4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

### 1. Fetch the Transactions for the last 90 days

In [ ]:
# YOUR CODE HERE
# Get transactions for institution for specific date range
start_date= '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-90))
end_date = '{:%Y-%m-%d}'.format(datetime.datetime.now())

# Get transactions for date range
transaction_response = client.Transactions.get(access_token,start_date,end_date)

### 2. Print the categories for each transaction

In [ ]:
# YOUR CODE HERE
for transactions in transaction_response['transactions']:
     print(json.dumps(transactions['category'], indent=None, sort_keys=True))


### 3. Create a new DataFrame using the following fields from the JSON transaction data: date, name, amount, category. 

(For categories with more than one label, just use the first category label in the list)

In [ ]:
# YOUR CODE HERE
df = pd.DataFrame(transaction_response['transactions'], columns = ['date', 'name','amount','category'])
df[['category1','category2','category3']] = pd.DataFrame(df.category.values.tolist(), index= df.index)
df_new=df.drop(columns=['category','category2','category3'])
df_new

### 4. Convert the data types to the appropriate types 

(i.e. datetimeindex for the date and float for the amount)

In [ ]:
# YOUR CODE HERE
df_new.dtypes

In [ ]:
df_new['date'] = pd.to_datetime(df_new['date'])
df_new['amount'] = df_new['amount'].astype('float')
df_new.dtypes

---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:
1. Determine the previous year's gross income and print the results
2. Determine the current monthly income and print the results
3. Determine the projected yearly income and print the results

In [ ]:
# YOUR CODE HERE
# Get income for date range
income_response = client.Income.get(access_token)
income_response

In [ ]:
 1. Determine the previous year's gross income and print the results
print(f" The previous years gross income is ${income_response['income']['last_year_income_before_tax']}.")
# 2. Determine the current monthly income and print the results
print(f" The current monthly income is ${income_response['income']['income_streams'][0]['monthly_income']}.")
# 3. Determine the projected yearly income and print the results
print(f" The projected yearly income is ${income_response['income']['projected_yearly_income_before_tax']}.")

---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)
2. Generate a bar chart with the number of transactions for each category 
3. Calculate the expenses per month
4. Plot the total expenses per month

### Calculate the expenses per category

In [ ]:
# YOUR CODE HERE# 1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)
#df_new_positive=df_new[df_new['amount']>0]
#df_new_positive
total_spending_per_category = df_new.groupby('category1')['amount'].sum()
total_spending_per_category.plot(kind='pie',title='Total spending per category',legend=True)
total_spending_per_category

In [ ]:
#2. Generate a bar chart with the number of transactionstotal spending per category for each category
number_spending_per_category = df_new.groupby('category1')['amount'].count()
number_spending_per_category.plot(kind='bar',title='Number of transactionstotal spending per category',legend=True)

### Calculate the expenses per month

In [ ]:
# YOUR CODE HERE
# 3. Calulate the expenses per month
df_new.index=pd.to_datetime(df_new['date'])
df_new

In [ ]:
df_new_grouped_by_month=df_new.groupby([df_new.index.month]).sum()
df_new_grouped_by_month

In [ ]:
# 4. Plot the total expenses per month
df_new_grouped_by_month.plot(kind='bar',title='Total expenses per month')